packages

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt

from PIL import Image
import cv2

import torch
from torch import nn
from torch.functional import F
import torchvision
from torchvision import transforms

from importlib import reload

In [2]:
import cambridge
reload(cambridge)

import models.posenet
reload(models.posenet)

import criterion
reload(criterion)

<module 'criterion' from '/home/bj/workspace/teaching/PoseNet/criterion.py'>

meta-parameters

In [3]:
batch_size = 128 
learning_rate = 1e-4
num_epochs = 100
image_height = 360
image_width = 480

device

In [4]:
# cuda or mps
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f"using {device} device")

using cuda device


logger

In [5]:
# logger
class AverageMeter():
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

transform

In [6]:
# basic transform
transform = transforms.Compose([
    transforms.Resize((image_height, image_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

dataset

In [7]:
# load dataset
dataset_root = '/home/bj/Datasets/Cambridge/Street'
train_set = cambridge.CambridgeDataset(dataset_root, 'train', transform=transform)

# split train and val set
split_ratio = 0.8
seed = 42
torch.manual_seed(seed)
train_set, val_set = torch.utils.data.random_split(train_set, 
                                                   [int(len(train_set)*split_ratio), 
                                                    len(train_set)-int(len(train_set)*split_ratio)])


In [8]:
test_set = cambridge.CambridgeDataset(dataset_root, 'test', transform=transform)

dataloader

In [9]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

model

In [10]:
model = models.posenet.PoseNet()
model.to(device)

PoseNet(
  (backbone): Sequential(
    (0): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_size=(1, 1), strid

optimizer

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

training loop

In [12]:
loss_meter = AverageMeter()
tr_loss_meter = AverageMeter()
rot_loss_meter = AverageMeter()

train_loss_log = []
train_tr_loss_log = []
train_rot_loss_log = []
val_loss_log = []
val_tr_loss_log = []
val_rot_loss_log = []

In [13]:
total_step = len(train_loader)
visualize_step = total_step // 10

In [14]:
best_val_loss = 1e10

for epoch in range(num_epochs):

    for param_group in optimizer.param_groups:
        print('learing rage: ', param_group['lr'])

    # train
    model.train()
    print ('------------------- Train: Epoch [{}/{}] -------------------'.format(\
        epoch+1, num_epochs) )

    loss_meter.reset()
    tr_loss_meter.reset()
    rot_loss_meter.reset()

    for i, (image, target_tr, target_rot) in enumerate(train_loader):
        image = image.to(device)
        target_tr = target_tr.to(device)
        target_rot = target_rot.to(device)

        # Forward pass
        pred_tr, pred_rot = model(image)
        loss, tr_loss, rot_loss = criterion.compute_pose_loss(pred_tr, pred_rot, target_tr, target_rot)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # logging
        loss_meter.update(loss.item(), image.size()[0] )
        tr_loss_meter.update(tr_loss.item(), image.size()[0] )
        rot_loss_meter.update(rot_loss.item(), image.size()[0] )

        if (i+1) % visualize_step == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Tr Loss: {:.4f}, Rot Loss: {:.4f}'.format(\
                epoch+1, num_epochs, i+1, total_step, loss.item(), tr_loss.item(), rot_loss.item() ) )

    print ('==> Train loss: {:.4f}'.format(loss_meter.avg) )

    train_loss_log.append(loss_meter.avg)
    train_tr_loss_log.append(tr_loss_meter.avg)
    train_rot_loss_log.append(rot_loss_meter.avg)

    # val
    model.eval() 
    print ('------------------- Val: Epoch [{}/{}] -------------------'.format(\
        epoch+1, num_epochs) ) 
    
    loss_meter.reset()
    tr_loss_meter.reset()
    rot_loss_meter.reset()

    for i, (image, target_tr, target_rot) in enumerate(val_loader):
        image = image.to(device)
        target_tr = target_tr.to(device)
        target_rot = target_rot.to(device)

        # Forward pass
        pred_tr, pred_rot = model(image)
        loss, tr_loss, rot_loss = criterion.compute_pose_loss(pred_tr, pred_rot, target_tr, target_rot)

        # logging
        loss_meter.update(loss.item(), image.size()[0] )
        tr_loss_meter.update(tr_loss.item(), image.size()[0] )
        rot_loss_meter.update(rot_loss.item(), image.size()[0] )

        if (i+1) % visualize_step == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Tr Loss: {:.4f}, Rot Loss: {:.4f}'.format(\
                epoch+1, num_epochs, i+1, total_step, loss.item(), tr_loss.item(), rot_loss.item() ) )
            
    print ('==> Val loss: {:.4f}'.format(loss_meter.avg) )

    val_loss_log.append(loss_meter.avg)
    val_tr_loss_log.append(tr_loss_meter.avg)
    val_rot_loss_log.append(rot_loss_meter.avg)

    # save model
    if loss_meter.avg < best_val_loss:
        best_val_loss = loss_meter.avg
        if not os.path.exists('checkpoint'):
            os.makedirs('checkpoint')
        torch.save(model.state_dict(), 'checkpoint/best.pth')


learing rage:  0.0001
------------------- Train: Epoch [1/100] -------------------
Epoch [1/100], Step [1/19], Loss: 6352.8789, Tr Loss: 6236.3398, Rot Loss: 0.3885
Epoch [1/100], Step [2/19], Loss: 6130.8013, Tr Loss: 6089.3525, Rot Loss: 0.1382
Epoch [1/100], Step [3/19], Loss: 5437.2368, Tr Loss: 5397.8633, Rot Loss: 0.1312
Epoch [1/100], Step [4/19], Loss: 5646.1914, Tr Loss: 5606.0059, Rot Loss: 0.1340
Epoch [1/100], Step [5/19], Loss: 5534.5347, Tr Loss: 5498.5972, Rot Loss: 0.1198
Epoch [1/100], Step [6/19], Loss: 5939.9590, Tr Loss: 5896.1953, Rot Loss: 0.1459
Epoch [1/100], Step [7/19], Loss: 5119.8530, Tr Loss: 5077.7808, Rot Loss: 0.1402
Epoch [1/100], Step [8/19], Loss: 5122.9961, Tr Loss: 5085.2949, Rot Loss: 0.1257
Epoch [1/100], Step [9/19], Loss: 5177.0425, Tr Loss: 5141.0957, Rot Loss: 0.1198
Epoch [1/100], Step [10/19], Loss: 3750.2288, Tr Loss: 3710.0703, Rot Loss: 0.1339
Epoch [1/100], Step [11/19], Loss: 4139.4648, Tr Loss: 4099.1465, Rot Loss: 0.1344
Epoch [1/100]

OutOfMemoryError: CUDA out of memory. Tried to allocate 508.00 MiB. GPU 0 has a total capacty of 47.53 GiB of which 190.19 MiB is free. Including non-PyTorch memory, this process has 47.14 GiB memory in use. Of the allocated memory 46.31 GiB is allocated by PyTorch, and 500.08 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF